In [ ]:
## Written by Hong-Kang Tian at BASF (Rochester Hills, MI), 2018/8
## Use for auto-calculating and plotting the Magnetic field vs. signal and calculate the H/2 for all files at once
## Put all the raw files (.vsm) under the same folder (specify the path in the input secion)
## There will be output plots that show the raw plot, after shifing, and after substracting the reference linear line
## Also, there will be two output files, which include a summarize file of Ms, 1/2 H, and a csv file for further plotting    
## in the input section, Fitting range of linear line and curve can be changed if necessary
## the initial guess of mu/kT is just the first guess, the fitting process will find the best solution automatically

import csv
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit 
import sympy as sp
from scipy.optimize import fsolve
from pandas import DataFrame
import pandas as pd


#### Input section

Folder_Path = '/Users/HONG-KANG/Documents/BASF/Python Code/MS/' #This path will be different in MacOX and Windows  
Plotting_data_output_FileName = 'Plotting_output.csv'           # Output file name of the plotting data 
Ms_Hf_output_FileName = 'Ms_Hf_output.csv'                      # Output file name of summary of Ms and 1/2 H
Fitting_linear_range = 20          # The range of data from the maximum field that used to fit the linear line of reference
Fitting_curve_range = 30
Inital_guess_mu_kT = 0.003       
# This fitting process will find a best mu/kT, but if fitted curve doesn't look good, this value can be modified

#### Input section ends


# Record the file name separately in the folder
File_name_list = os.listdir(Folder_Path)
File_name_list.sort()

# Rewrite the output files to strat from blank
with open(Plotting_data_output_FileName, 'w', newline='') as Plotting_file:  
    writer = csv.writer(Plotting_file)         
with open(Ms_Hf_output_FileName, 'w', newline='') as Ms_Hf_file:  
    writer = csv.writer(Ms_Hf_file)
    writer.writerow(['Sample name','Ms (EMU/g)','1/2 H (Oe)'])

All_Final_Field = []
All_Final_Signal = []
All_Final_Name = []
All_Final_xy = []
# Operate the files under the folder one by one 
for Open_File in File_name_list:
    File_order = File_name_list.index(Open_File)      # let the file order starts from 1
    with open(Plotting_data_output_FileName, 'a', newline='') as Plotting_file:  
        writer = csv.writer(Plotting_file)   
    
    All_Raw_Data=[]
    Field = []
    Singal_x_Normalized = []
    
    # Open the raw data file
    with open(Folder_Path+Open_File,'r', newline='',encoding='utf-8') as Raw_Data:       
        Headers_Reader = csv.reader(Raw_Data,delimiter='=')       
        for Headers in Headers_Reader:
            if Headers[0] == 'Sample Mass':
                Sample_Mass = (float(Headers[1]))
                break
        Content_Reader = csv.reader(Raw_Data,delimiter=';')  
        for Content in Content_Reader: 
            if len(Content)>1:
                All_Raw_Data.append(Content)
                Field.append(float(Content[2]))
                Singal_x_Normalized.append(float(Content[3])/Sample_Mass)
    
    # Find the position of the minimum of field, only record the data from positive to negative field
    Filed_min_index = Field.index(min(Field))
    del(Field[Filed_min_index+1:])
    del(Singal_x_Normalized[Filed_min_index+1:])
    
    # Fine the position when field = 0, record the intercept, which is the shift
    Field_0_index = Field.index(0)
    Intercept_Signal = Singal_x_Normalized[Field_0_index]
    
    # Shifting the raw signal/weight data
    Singal_x_Normalized_shifted = []
    for i in Singal_x_Normalized:
        Singal_x_Normalized_shifted.append(i-Intercept_Signal)
        
    # Fit the raw data 
    fit_Signal = np.polyfit(Field[0:Fitting_linear_range], Singal_x_Normalized_shifted[0:Fitting_linear_range],1)   
    fit_fn_Signal = np.poly1d(fit_Signal)
    Fitting_line_y = fit_fn_Signal
    
    # Making the intercept of the fitting line to be zero
    #fit_fn_Signal[0] = 0
    
    # Substract the slope of fitting line from the normalized signal
    Final_Signal = []
    for i in range(len(Field)):
        Final_Signal.append(Singal_x_Normalized_shifted[i]-fit_fn_Signal(Field[i])+fit_Signal[1]) 
    
    # Define the fitting function 
    def func(x,Ms,mu_kT): 
        return Ms*(np.cosh(mu_kT*x)/(np.sinh(mu_kT*x))-1/(mu_kT*x))
    
    # Define new x-y data whose data of zero field will be deleted
    Field_substract_zero = []
    Signal_substract_zero = []
    for i in Field:
        Field_substract_zero.append(i)
    for i in Final_Signal:
        Signal_substract_zero.append(i)
     
    # Delete the zero point of Field to aviod the error when using coth() function
    Zero_field_index = Field.index(0)
    del Field_substract_zero[Zero_field_index]
    del Signal_substract_zero[Zero_field_index]
    
    # Fit the data point by a initial guess of signal
    Ms_initial_guess = max(Singal_x_Normalized_shifted)
    popt, pcov = curve_fit(func,\
                           Field_substract_zero[
                               int(1/2*len(Field_substract_zero)-Fitting_curve_range):
                               int(1/2*len(Field_substract_zero)+Fitting_curve_range)],
                           Signal_substract_zero[
                               int(1/2*len(Field_substract_zero)-Fitting_curve_range):
                               int(1/2*len(Field_substract_zero)+Fitting_curve_range)], 
                           p0=[Ms_initial_guess,
                           Inital_guess_mu_kT])
    
    # Solve the fitted function to find the position of 1/2 H (field) that gives 1/2 of Ms (signal)
    def final_func(H):
        return popt[0]*(np.cosh(popt[1]*H)/(np.sinh(popt[1]*H))-1/(popt[1]*H))-1/2*popt[0]
    r = fsolve(final_func,500)
    
    # Write all the Ms and 1/2 H data into the output Ms_Hf file
    with open(Ms_Hf_output_FileName, 'a', newline='') as Ms_Hf_file:  
        writer = csv.writer(Ms_Hf_file)
        writer.writerow([Open_File,popt[0],r[0]])
     
    # Preparing the output plotting data format
    All_Final_Field.append(Field)
    All_Final_Signal.append(Final_Signal)
    All_Final_Name.append(Open_File)
    All_Final_xy.append('Normalized Signal (EMU/g)')
    
    print(Open_File)   
    print('Ms = ' , popt[0])
    print('mu/kT = ',popt[1])
    print('1/2H = ',r[0])
    plt.plot(Field,Singal_x_Normalized)
    plt.title('Raw_plot')
    plt.xlabel('Field (Oe)')
    plt.ylabel('Normalized Signal (EMU/g)')
    
    plt.show()    
    plt.plot(Field, Singal_x_Normalized_shifted,\
             Field[0:Fitting_linear_range],Fitting_line_y(Field[0:Fitting_linear_range]),lw=2)
    plt.title('Raw_plot_shifted')
    plt.xlabel('Field (Oe)')
    plt.ylabel('Normalized Signal (EMU/g)')
    plt.legend(labels=['Data Points','Fitted Line'])
    plt.show()    
    
    plt.plot(Field, Final_Signal,Field_substract_zero,func(np.array(Field_substract_zero),popt[0],popt[1]) )
    plt.title('Final_Signal')
    plt.xlabel('Field (Oe)')
    plt.ylabel('Normalized Signal (EMU/g)')
    plt.legend(labels=['Data Points','Fitted Curve'])
    plt.show()   

All_Final_Name.insert(0,'')
All_Final_xy.insert(0,'Field (oE)')
with open(Plotting_data_output_FileName, 'a', newline='') as Plotting_file:  
    writer = csv.writer(Plotting_file)
    writer.writerow(All_Final_Name)
    writer.writerow(All_Final_xy)    
       
for i in range(len(All_Final_Field[0])):
    Recorded_data =[]
    Recorded_data.append(All_Final_Field[0][i])
    for j in range(len(All_Final_Field)):
        Recorded_data.append(All_Final_Signal[j][i])
    
    with open(Plotting_data_output_FileName, 'a', newline='') as Plotting_file:  
        writer = csv.writer(Plotting_file)
        writer.writerow(Recorded_data)